# Expressions in SysML

Expressions in SysML can turn into deeply nested trees. This notebook shows how to work with them.

In [ ]:
import json
from importlib import resources as lib_resources
import pymbe.api as pm

from pymbe.model import Element

from uuid import uuid4

from pymbe.query.metamodel_navigator import \
    get_effective_basic_name, get_effective_lower_multiplicity, get_effective_upper_multiplicity, get_most_specific_feature_type

from pymbe.model_modification import (
    build_from_classifier_pattern,
    build_from_feature_pattern,
    assign_value_by_literal_expression,
    assign_multiple_values_with_fre,
    create_element_data_dictionary,
    build_from_expression_pattern,
    build_from_parameter_pattern,
    assign_feature_value_to_expression,
)

from pymbe.text_concrete_syntax import serialize_sysml_package

In [ ]:
library_model = None

with lib_resources.path("pymbe.static_data", "SystemsLibrary.json") as lib_data1:
    with lib_resources.path("pymbe.static_data", "KernelLibraryExpanded.json") as lib_data2:
            library_model = pm.Model.load_from_mult_post_files([lib_data1, lib_data2])

In [ ]:
empty_model = pm.Model(elements={})

package_model_namespace_data = {
    "aliasIds": [],
    "isImpliedIncluded": False,
    "@type": "Namespace",
    "@id": str(uuid4()),
    "ownedRelationship": [],
}

package_model_data = {
    "name": "SysML Value Model",
    "declaredName": "SysML Value Model",
    "isLibraryElement": False,
    "filterCondition": [],
    "ownedElement": [],
    "owner": {},
    "@type": "Package",
    "@id": str(uuid4()),
    "ownedRelationship": [],
}

new_ns = Element.new(data=package_model_namespace_data, model=empty_model)

new_package = Element.new(data=package_model_data, model=empty_model)

empty_model.reference_other_model(library_model)

## Generating Expressions

Elements in an expression tree reference each other in a cyclical way, where nodes need owners but when owners are serialized, the serializations of their children are included. To get around this, the generation of serialization is held off until the expression is fully built. First parameters and the expressions relating them are generated, then values are assigned to inputs as needed to build up the expression tree. The value assignment call also sets up ownership of the expressions within the model.

### Example 1: Simple addition and subtraction

Do a binary operation with the + symbol and then the second part of the expression is a subtraction.

In [ ]:
sf_ns = [
        library_model_ns
        for library_model_ns in library_model.ownedElement
        if library_model_ns.throughOwningMembership[0].declaredName == "ScalarFunctions"
    ][0]

In [ ]:
plus_func = sf_ns.throughOwningMembership[0].throughOwningMembership[1]

In [ ]:
minus_fuc = sf_ns.throughOwningMembership[0].throughOwningMembership[2]

Create literal expressions, which have only return parameters, first so they can be added to the expression tree for higher-order operations.

In [ ]:
lr1_return_para = build_from_parameter_pattern(
        name="result",
        model=empty_model,
        specific_fields={},
        feature_type=None,
        direction="out",
        metatype="Feature",
        returning_parameter=True,
    )
lr1_return_para

In [ ]:
lr1 = build_from_expression_pattern(
    model=empty_model,
    specific_fields={"value": 3.4},
    metatype="LiteralRational",
    in_paras=[],
    return_para=lr1_return_para
)
lr1

In [ ]:
lr2_return_para = build_from_parameter_pattern(
        name="result",
        model=empty_model,
        specific_fields={},
        feature_type=None,
        direction="out",
        metatype="Feature",
        returning_parameter=True,
    )
lr2_return_para

In [ ]:
lr2 = build_from_expression_pattern(
    model=empty_model,
    specific_fields={"value": 6.7},
    metatype="LiteralRational",
    in_paras=[],
    return_para=lr1_return_para
)
lr2

In [ ]:
lr3_return_para = build_from_parameter_pattern(
        name="result",
        model=empty_model,
        specific_fields={},
        feature_type=None,
        direction="out",
        metatype="Feature",
        returning_parameter=True,
    )
lr3_return_para

In [ ]:
lr3 = build_from_expression_pattern(
    model=empty_model,
    specific_fields={"value": 11.1},
    metatype="LiteralRational",
    in_paras=[],
    return_para=lr3_return_para
)
lr3

In [ ]:
adder_x_para = build_from_parameter_pattern(
    name="x",
    model=empty_model,
    specific_fields={},
    feature_type=None,
    direction="in",
    metatype="Feature",
    returning_parameter=False,
)

In [ ]:
adder_y_para = build_from_parameter_pattern(
    name="y",
    model=empty_model,
    specific_fields={},
    feature_type=None,
    direction="in",
    metatype="Feature",
    returning_parameter=False,
)

In [ ]:
adder_return_para = build_from_parameter_pattern(
    name="result",
    model=empty_model,
    specific_fields={},
    feature_type=None,
    direction="out",
    metatype="Feature",
    returning_parameter=True,
)

In [ ]:
subtractor_x_para = build_from_parameter_pattern(
    name="x",
    model=empty_model,
    specific_fields={},
    feature_type=None,
    direction="in",
    metatype="Feature",
    returning_parameter=False,
)

In [ ]:
subtractor_y_para = build_from_parameter_pattern(
    name="y",
    model=empty_model,
    specific_fields={},
    feature_type=None,
    direction="in",
    metatype="Feature",
    returning_parameter=False,
)

In [ ]:
subtractor_return_para = build_from_parameter_pattern(
    name="result",
    model=empty_model,
    specific_fields={},
    feature_type=None,
    direction="out",
    metatype="Feature",
    returning_parameter=True,
)

First build the second part of the expression (the subtraction).

In [ ]:
subtractor = build_from_expression_pattern(
    model=empty_model,
    specific_fields={"operator": get_effective_basic_name(minus_fuc)},
    metatype="OperatorExpression",
    in_paras=[subtractor_x_para, subtractor_y_para],
    return_para=subtractor_return_para
)

In [ ]:
assign_feature_value_to_expression(
    target_feature=subtractor_x_para,
    expr=lr2,
    model=empty_model
)

In [ ]:
assign_feature_value_to_expression(
    target_feature=subtractor_y_para,
    expr=lr3,
    model=empty_model
)

With the subtractor side built, now add in the adder side.

In [ ]:
assign_feature_value_to_expression(
    target_feature=adder_x_para,
    expr=lr1,
    model=empty_model
)

In [ ]:
assign_feature_value_to_expression(
    target_feature=adder_y_para,
    expr=subtractor,
    model=empty_model
)

In [ ]:
adder = build_from_expression_pattern(
    model=empty_model,
    specific_fields={"operator": get_effective_basic_name(plus_func)},
    metatype="OperatorExpression",
    in_paras=[adder_x_para, adder_y_para],
    return_para=adder_return_para
)

In [ ]:
subtractor

In [ ]:
adder